In [1]:
import pandas as pd
df = pd.read_csv('D:\SoB\MainSoB.csv',sep=",",header=0, index_col=False)
df = df.drop(df.columns[0], axis=1)

In [2]:
df.head()

,Year,State.Code,State.Abbr,County.Code,County.Name,Commodity.Code,Commodity.Name,Insurance.Plan.Code,Insurance.Plan.Abbr,Coverage.Category,...,Net.Planted.Quantity,Net.Endorsed.Acres,Liability,Total.Premium,Subsidy,State/Provate.Subsidy,Additional.Subsidy,EFA.Premium.Discount,Indemnity.Amount,Loss.Ratio
0,1989,1,AL,0,All Other Counties,9999,All Other Crops,90,APH,E,...,0.0,0,0.0,0.0,0.0,0,0,0,0,0.00
1,1989,1,AL,999,All Other Counties,9999,All Other Crops,90,APH,E,...,0.0,0,0.0,0.0,0.0,0,0,0,0,0.00
2,1989,1,AL,999,All Other Counties,9999,All Other Crops,90,APH,E,...,1147.0,0,206577.0,13216.0,3988.0,0,0,0,36662,2.77
3,1989,1,AL,999,All Other Counties,9999,All Other Crops,90,APH,E,...,413.0,0,95934.0,8352.0,2473.0,0,0,0,3720,0.45
4,1989,1,AL,999,All Other Counties,9999,All Other Crops,90,APH,E,...,180.0,0,7207.0,1892.0,319.0,0,0,0,4983,2.63


In [3]:
#Drop Irrelevant Rows
print(len(df))
Territories_To_Drop = [2, 11, 15, 72, 78]
df = df[~df['State.Code'].isin(Territories_To_Drop)]
#Drops Alaska, DC, Hawaii, Puerto Rico and the Virgin Islands
print(len(df))
df = df[~(df['Commodity.Code']==9999)]
# Drop the "Other" Commodity
df = df[df['Quantity.Type'].str.strip()=='Acres']
# Drops Trees, Clams, Raisins, Non/Plant
print(len(df))

3396795
3395556
3122732


In [4]:
# Function to add leading 0 to a string if its length is smaller than the maximum length in the column
def add_leading_zero(column):
    max_length = df[column].astype(str).str.len().max()
    return df[column].astype(str).apply( lambda string: string.zfill(max_length) if len(string) < max_length else string)

In [5]:
#Create FIPS out of state and county components
df['FIPS']=add_leading_zero('State.Code')+add_leading_zero('County.Code')
df.tail()

,Year,State.Code,State.Abbr,County.Code,County.Name,Commodity.Code,Commodity.Name,Insurance.Plan.Code,Insurance.Plan.Abbr,Coverage.Category,...,Net.Endorsed.Acres,Liability,Total.Premium,Subsidy,State/Provate.Subsidy,Additional.Subsidy,EFA.Premium.Discount,Indemnity.Amount,Loss.Ratio,FIPS
3396780,2021,56,WY,43,Washakie,88,"Pasture,Rangeland,Forage",13,RI,A,...,0,571638.0,101677.0,51854.0,0,0,0,88641,0.87,56043
3396781,2021,56,WY,43,Washakie,39,Sugar Beets,90,APH,A,...,0,5702216.0,252288.0,162528.0,0,0,0,117923,0.47,56043
3396782,2021,56,WY,43,Washakie,39,Sugar Beets,90,APH,A,...,0,615971.0,30462.0,17972.0,0,0,0,114589,3.76,56043
3396783,2021,56,WY,43,Washakie,39,Sugar Beets,90,APH,A,...,0,1299678.0,70551.0,41626.0,0,0,0,0,0.00,56043
3396784,2021,56,WY,43,Washakie,39,Sugar Beets,90,APH,A,...,0,55651.0,3319.0,1826.0,0,0,0,0,0.00,56043


In [6]:
#ID is Year x County tuple. Use FIPS to get what we mean when we say County
df['ID']=df['Year'].astype(str)+'_'+df['FIPS']
print(df['ID'].nunique())
CommodityCombinations=df.groupby('ID')['Commodity.Code'].agg(frozenset).value_counts()
#Gets each combination of remaining commodity codes for initial inspection. 
#No immediate favorites. Will have missing values problem. But can minimize it?

66585


In [8]:
CommodityCombinations.head(20)
# Convert the result to a DataFrame
result_df = pd.DataFrame({'CommodityCombination': CommodityCombinations.index, 'Count': CommodityCombinations.values})

# Export the result to a CSV file
result_df.to_csv('commodity_combinations.csv', index=False)

In [9]:
# Generate List of Overlapping Commodities

#import pandas as pd

# Step 1: Group the DataFrame by 'ID' and aggregate 'Commodity' as a set
grouped = df.groupby('ID')['Commodity.Code'].agg(set).reset_index()

# Step 2: Create a dictionary to store commodity frequency counts
commodity_counts = {}

# Step 3: Iterate over rows and update commodity count dictionary
for _, row in grouped.iterrows():
    commodities = row['Commodity.Code']
    for commodity in commodities:
        if commodity in commodity_counts:
            commodity_counts[commodity] += 1
        else:
            commodity_counts[commodity] = 1

# Step 4: Sort commodities based on frequency count in descending order
sorted_commodities = sorted(commodity_counts, key=commodity_counts.get, reverse=True)

# Step 5: Select commodities with shared IDs while limiting missing values
selected_commodities = []
selected_ids = set()

for commodity in sorted_commodities:
    # Get IDs for the current commodity
    ids = set(grouped[grouped['Commodity.Code'].apply(lambda x: commodity in x)]['ID'])
    
    # Find common IDs with previously selected commodities
    common_ids = ids.intersection(selected_ids)
    
    # Check if including this commodity increases the length of selected list
    if len(common_ids) < len(selected_commodities):
        continue
    
    # Update selected IDs and commodities
    selected_ids.update(ids)
    selected_commodities.append(commodity)

In [10]:
#Will check the above against which commodities have significant total presence in the FCIP
#Generate list of most insured crops by total acres
totaled_df = df.groupby('Commodity.Code')['Net.Planted.Quantity'].sum().sort_values(ascending=False).reset_index()
totaled_df.to_csv('TotalInsuredQuantity.csv',index=True)

In [11]:
OverlappingAlgorithmCommodities = selected_commodities[:20]
HighlyInsuredCommodities = totaled_df['Commodity.Code'].tolist()[:20]
temp = set(OverlappingAlgorithmCommodities+HighlyInsuredCommodities)# List of Commodity.Codes

# Set 'Commodity.Code' as the index and select 'Commodity.Name' column
code_name_df = df.set_index('Commodity.Code')['Commodity.Name']

# Create a dictionary using the code_name_df DataFrame
code_name_dict = code_name_df.to_dict()

# Filter the dictionary for the specified codes
filtered_dict = {code: code_name_dict.get(code) for code in temp}

In [12]:
labels = [filtered_dict.get(item) for item in OverlappingAlgorithmCommodities]
OverlappingAlgorithmComms= pd.DataFrame(zip(OverlappingAlgorithmCommodities,labels), columns=['Values','Label'])

In [13]:
OverlappingAlgorithmComms

,Values,Label
0,41,Corn
1,11,Wheat
2,81,Soybeans
3,16,Oats
4,51,Grain Sorghum
5,91,Barley
6,21,Cotton
7,88,"Pasture,Rangeland,Forage"
8,33,Forage Production
9,78,Sunflowers


In [14]:
labels = [filtered_dict.get(item) for item in HighlyInsuredCommodities]
HighlyInsuredComms= pd.DataFrame(zip(HighlyInsuredCommodities,labels), columns=['Values','Label'])

In [15]:
HighlyInsuredComms

,Values,Label
0,41,Corn
1,81,Soybeans
2,11,Wheat
3,88,"Pasture,Rangeland,Forage"
4,21,Cotton
5,51,Grain Sorghum
6,48,RANGELAND
7,91,Barley
8,33,Forage Production
9,18,Rice


In [16]:
#Attach the Rank of most insured crops to the list of most overlapping crops
HighlyInsuredComms['Rank'] = HighlyInsuredComms.index+1
temp= pd.DataFrame(HighlyInsuredComms[['Values','Rank']])
merged_comms = OverlappingAlgorithmComms.merge(temp, on='Values', how='left')
merged_comms.rename(columns={'Values':'Commodity.Code','Label':'Commodity.Name','Rank':'Quantity.Insured.Rank'},inplace=True)
merged_comms

,Commodity.Code,Commodity.Name,Quantity.Insured.Rank
0,41,Corn,1.0
1,11,Wheat,3.0
2,81,Soybeans,2.0
3,16,Oats,17.0
4,51,Grain Sorghum,6.0
5,91,Barley,8.0
6,21,Cotton,5.0
7,88,"Pasture,Rangeland,Forage",4.0
8,33,Forage Production,9.0
9,78,Sunflowers,11.0


In [17]:
#Based on these results, I suggest Corn, Wheat, Soybeans, Grain Sorghum, and Barley as commonly insured staples for our analysis
#Cotton and Rangeland should also be included, tentatively, due to high available overlap
values_list=[41,11,81,51,88,91,21]
Final=merged_comms[merged_comms['Commodity.Code'].isin(values_list)]
Final.to_csv('SelectedCommodities.csv',index=True)
Final

,Commodity.Code,Commodity.Name,Quantity.Insured.Rank
0,41,Corn,1.0
1,11,Wheat,3.0
2,81,Soybeans,2.0
4,51,Grain Sorghum,6.0
5,91,Barley,8.0
6,21,Cotton,5.0
7,88,"Pasture,Rangeland,Forage",4.0


In [17]:
# Counterarguments
#Why only within county? Could assign each county itself and neighboring counties in order to have fewer missing commodities, more interesting divergence
#Food for thought for future